In [2]:
import torch
from transformer.flow import make_model, batch_size_fn, run_epoch

vocab_src = torch.load("./noie_full_6heads_2048ffsrc_vocab.pt")
vocab_tgt = torch.load("./noie_full_6heads_2048fftrg_vocab.pt")
model = make_model(len(vocab_src), len(vocab_tgt),
                        n=5, d_model=768,
                        d_ff=1200, h=6,
                        dropout=.1)
device = torch.device('cpu')
model.load_state_dict(torch.load('./noie_full_1200ff_6heads_r10_epoch0.pt', map_location=device))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [3]:
model.eval()

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadAttention(
          (linears): ModuleList(
            (0): Linear(in_features=768, out_features=768, bias=True)
            (1): Linear(in_features=768, out_features=768, bias=True)
            (2): Linear(in_features=768, out_features=768, bias=True)
            (3): Linear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PointerwiseFeedforward(
          (w_1): Linear(in_features=768, out_features=1200, bias=True)
          (w_2): Linear(in_features=1200, out_features=768, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
         

In [4]:
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel


def get_arg_tokenizer():
    tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
    max_key = max(tokenizer.encoder.values())
    add_toks = ['argonestart', 'argoneend', 'relstart', 'relend', 'argtwostart', 'argtwoend']
    word_piece = '</w>'
    for ix, tok in enumerate(add_toks):
        tokenizer.encoder[tok + word_piece] = max_key + 1 + ix
        tokenizer.decoder[max_key + 1 + ix] = tok + word_piece
        tokenizer.cache[tok] = tok + word_piece
    return tokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
# with open('./all.txt', encoding='UTF-8') as f:
#         test_data = f.read()
# a = test_data.split('\n')
# a    

In [6]:
tok = get_arg_tokenizer()

In [7]:
from transformer.greedy import greedy_decode
from torch.autograd import Variable

In [8]:
sent = tok.tokenize('Instead , much of numerical analysis is concerned with obtaining approximate solutions while maintaining reasonable bounds on errors .')
src = torch.LongTensor([[vocab_src.stoi[w] for w in sent]])
print(src)

tensor([[ 1038,     3,   477,     5,  6930,  1411,  1931,    10,  4203,    16,
         10199, 13278,  5011,    88,  6939, 11797,  9571,    17,  8944,     4]])


In [9]:
src = Variable(src)
src_mask = (src != vocab_src.stoi["<blank>"]).unsqueeze(-2)
src_mask

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]],
       dtype=torch.uint8)

In [10]:
out = greedy_decode(model, src, src_mask, 
                    max_len=60, start_symbol=vocab_tgt.stoi["<s>"])
print("Argument and relation boundaries:", end="\t")
trans = "<s> "
for i in range(1, out.size(1)):
    sym = vocab_tgt.itos[out[0, i]]
    if sym == "</s>": break
    trans += sym + " "
print(trans)

Argument and relation boundaries:	<s> argonestart</w> much</w> of</w> numer ical</w> analysis</w> argoneend</w> relstart</w> is</w> relend</w> argtwostart</w> concerned</w> with</w> obtaining</w> approximate</w> solutions</w> argtwoend</w> 
